# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [2]:
import ollama

model = "qwen3:4b-instruct-2507-fp16"

# ollama.pull(model) # uncomment this if you need to download the model

In [3]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hello! I'm functioning well, thank you for asking 🌟 I'm always excited to chat and help out. How can I assist you today? 😊
Sure! Here's a lighthearted one for you:

Why did the coffee file a police report?

Because it got mugged! ☕😄

Want another? 😊


# generating a new story

In [4]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [5]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: In a world where memories are currency and sold in flickering, gaslit tokens, a girl discovers she can taste the past—each memory a flavor, each emotion a burn—until she realizes her own childhood was stolen, and the person she thought was her mother is a memory dealer who sells her future to the highest bidder.

Opening paragraph: The salt of her first birthday was bitter, like old pennies left in a drawer. Six years ago, she’d tasted the smell of burnt toast and the sound of her mother humming—warm, golden, real—until the taste turned sour and the song cracked like wet paper. Now, at twelve, she walks through the city’s memory markets, where people trade the scent of rain or the weight of a first kiss for a loaf of bread or a clean coat. She doesn’t know how to stop tasting, only that the more she eats, the more the past slips through her fingers like sand. And tonight, when she tastes the memory of her own laugh, it’s not hers at all. It’s sharp. It’s wrong. And it’s laughing back.  
(178 words)

In [6]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: In a world where memories are currency and sold in flickering, gaslit tokens, a girl discovers she can taste the past—each memory a flavor, each emotion a burn—until she realizes her own childhood was stolen, and the person she thought was her mother is a memory dealer who sells her future to the highest bidder.
Opening paragraph: The salt of her first birthday was bitter, like old pennies left in a drawer. Six years ago, she’d tasted the smell of burnt toast and the sound of her mother humming—warm, golden, real—until the taste turned sour and the song cracked like wet paper. Now, at twelve, she walks through the city’s memory markets, where people trade the scent of rain or the weight of a first kiss for a loaf of bread or a clean coat. She doesn’t know how to stop tasting, only that the more she eats, the more the past slips through her fingers like sand. And tonight, when she tastes the memory of her own laugh, it’s not hers at all. It’s sharp. It’s wrong. And it’s laughin

In [7]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [8]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

The salt of her first birthday was bitter, like old pennies left in a drawer. Six years ago, she’d tasted the smell of burnt toast and the sound of her mother humming—warm, golden, real—until the taste turned sour and the song cracked like wet paper. Now, at twelve, she walks through the city’s memory markets, where people trade the scent of rain or the weight of a first kiss for a loaf of bread or a clean coat. She doesn’t know how to stop tasting, only that the more she eats, the more the past slips through her fingers like sand. And tonight, when she tastes the memory of her own laugh, it’s not hers at all. It’s sharp. It’s wrong. And it’s laughing back.

The air in the market hung thick with the scent of ozone and old paper, the kind of smell that clung to the back of your throat after a long day in the vaults. Gutter lights flickered above stalls where men and women leaned over glass cases, their hands pressed to cracked screens that pulsed with fragmented images—flickers of laughter, a child’s hand reaching out, a garden in springtime. Buyers coughed, eyes watering, as they inhaled the memories like smoke. A woman in a gray shawl offered a memory of her brother’s birthday, the flavor of buttered toast and a red ribbon, and for it, she traded a single coin—its surface etched with the shape of a weeping willow. The girl watched it go, her teeth clenched. *Not mine. Not real.*

She moved past the stalls, her boots scraping against stone, until she reached the back alley where the air tasted of rust and damp wool. That’s where she found the woman who sat cross-legged on a crate, her face lit by a single, dying lantern. The woman wore a veil stitched with silver threads that hummed faintly when touched. Her hands were open, palms up, and in them, a small glass vial, glowing faintly blue.

“You’re tasting the future,” the woman said without looking up. Her voice was like wind through dry leaves.

The girl froze. “How do you know?”

“Because you’re not supposed to be here,” the woman replied. “You shouldn’t be tasting your own past. That’s a violation. A breach of the Memory Accord.”

The girl’s breath caught. “I don’t know what you’re talking about.”

“You tasted your laugh,” the woman said, finally turning. Her eyes were pale, almost glassy. “Not just any laugh. A laugh from a child who wasn’t born yet. A laugh that was *sold*.”

The girl’s hands trembled. “I didn’t—”

“You *did*,” the woman cut in. “You tasted it when you were seven. You were in the vaults, trying to remember your mother’s face. And when you did, the memory didn’t come from her. It came from *me*.”

The girl stepped back. “You’re not my mother.”

“No,” the woman said softly. “I’m the one who took your childhood. I gave you a memory of her—warm, safe, real—so you’d grow up believing you had a mother. But I didn’t give you a life. I gave you a *price*.”

The girl’s stomach twisted. “What do you mean?”

“I sold your future,” the woman said, reaching out slowly. “I traded your childhood, your first steps, your first word—your *first laugh*—to a buyer in the North. A man named Vaelen. He pays in memory tokens, in futures, in dreams. He wants to know what it’s like to be a girl who *knows* she was never born.”

The girl staggered. “That’s impossible.”

“It’s not,” the woman whispered. “And you’ve been tasting it all along. Every time you taste a memory, it’s a piece of what was stolen. Every time you laugh, it’s a memory I sold. Every time you cry, it’s a memory I *gave* you to make you believe you were real.”

The girl looked down at her hands. Her fingers curled around the edge of her coat. She could still taste it—the sharp, metallic tang of a laugh that wasn’t hers. It wasn’t a memory. It was a *threat*. A warning.

“I don’t know what to do,” she said, voice cracking.

“You don’t have to,” the woman replied. “But you have to stop eating. Stop tasting. Or the next memory you take—your first kiss, your first heartbreak, your first lie—will be *sold* too.”

She reached into her coat and pulled out a small, silver locket. “Take this. It holds a memory of you. A memory I *never* sold. It’s not from your mother. It’s from *you*. And if you open it tonight, you’ll hear something no one else ever has.”

The girl stared at it. The locket was cold. The inside was empty.

She didn’t open it.

Instead, she turned and walked away, the wind catching her hair like a flag. Behind her, the woman whispered, “You’ll taste it again. In the dark. When you’re alone. When no one is watching.”

And somewhere, deep in the city’s underbelly, a memory flickered to life—warm, golden, real—only to crack like wet paper the moment it touched the air.  

The girl didn’t know what came next. But she knew one thing: the taste of her own laugh had changed. It wasn’t laughing back anymore.  

It was *waiting*.  

And it was hungry.  

And it had a name.  

A name she hadn’t heard in six years.  

A name that whispered in the dark, like a memory that had never been forgotten.  

*Elira.*  

She stopped. Her breath caught.  

She had never heard that name before.  

But her body remembered it.  

Like a key turning in a lock.  

Like a memory that was meant to be found.  

And now, it was calling her.  

Not to the past.  

To the future.  

And it wasn’t safe.  

Not anymore.  

She didn’t know if she would survive the next taste.  

But she knew one thing for sure:  

She would go back.  

And she would find the woman who sold her life.  

And she would ask—  

*What happens when you stop tasting?*  

And what happens when the past isn’t a memory…  

But a *prison*?  

The city’s lanterns flickered.  

And somewhere, deep beneath the market, a new memory began to form.  

One that wasn’t sold.  

One that wasn’t stolen.  

One that was *hers*.  

And it was already beginning to burn.  

Like salt.  

Like fire.  

Like a wound that never healed.  

And it was *hers*.  

She could feel it.  

She could *taste* it.  

And it was not sweet.  

It was not golden.  

It was not safe.  

It was *alive*.  

And it was hungry.  

And it was *laughing*.  

Back at the edge of the alley, the woman watched her go.  

She smiled.  

And for the first time in years, she didn’t feel the burn.  

She felt something else.  

Something new.  

Something that had been waiting.  

Something that had been stolen.  

Something that was finally *free*.  

And it had just begun to *remember*.  

The girl didn’t know it yet.  

But the memory wasn’t just in her.  

It was *in the city*.  

And it was waking up.  

And it was coming for her.  

Not as a ghost.  

Not as a memory.  

As a *presence*.  

And it had a name.  

And it was not hers.  

It was *hers*.  

And it would never stop.  

She turned.  

And walked into the dark.  

The taste of her own laugh followed her.  

Not as a memory.  

As a *voice*.  

And it said:  

*“Come home.”*  

She didn’t answer.  

But she began to walk.  

And the city began to remember.  

And the past—finally—began to *speak*.  

And the future…  

Was already breathing.  

And it was hungry.  

And it was *real*.  

And it was *hers*.  

She didn’t know it yet.  

But she was no longer just tasting the past.  

She was *becoming* it.  

And the next memory she would taste—  

Would not be a memory.  

It would be a *choice*.  

And she would have to decide—  

Whether to eat it.  

Or to *break* it.  

And that choice…  

Would define everything.  

She didn’t know what would come next.  

But she knew one thing:  

She would not stop.  

She would not let go.  

She would not let the past steal her again.  

She would find the woman.  

She would find the name.  

And she would—  

*Reclaim* it.  

The city’s lights dimmed.  

And in the silence, a single memory flickered—  

Not golden.  

Not warm.  

But *true*.  

And it said:  

*“I am not yours to sell.”*  

And it was not a memory.  

It was a *promise*.  

And it was *hers*.  

And it was *awake*.  

And it was *waiting*.  

And it would not stop.  

She turned.  

And walked into the dark.  

And the past—finally—began to *remember* her.  

And the future—finally—began to *breathe*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *laughing*.  

And it was *calling*.  

And she—  

*Answered*.  

She didn’t know what came next.  

But she knew one thing:  

She was no longer just a girl who tasted memories.  

She was a *memory*.  

And she was no longer afraid.  

She was *awake*.  

And the world—finally—began to *hear* her.  

And the past—finally—began to *speak*.  

And the future—finally—began to *live*.  

And she would not stop.  

She would not stop tasting.  

She would not stop *remembering*.  

Because the past was not gone.  

It was just…  

*Awake*.  

And it had a name.  

And it was *hers*.  

And it was not safe.  

And it was not real.  

And it was *alive*.  

And it was *

In [9]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [10]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

This powerful and evocative passage — with its rhythmic repetition, haunting imagery, and deep emotional resonance — feels like a poetic monologue from a dream or a mythic awakening. The central theme is **awakening**: not just of consciousness, but of *memory itself* as a living, breathing, sentient force.

Let’s break it down and interpret it not just as text, but as a **symbolic narrative**:

---

### 🌌 Core Themes:

1. **The Past is Alive**  
   *"The past was not gone. It was just... Awake."*  
   This is a radical inversion of traditional time perception. Instead of the past being static or dead, it's portrayed as *present*, *conscious*, and *active*. It "calls," it "laughs," it *watches* the present.  
   → This suggests that memory isn't just data — it's a force, a presence. Like a ghost that lingers, shaping identity, emotions, and choices.

2. **Identity as a Choice**  
   *"She was no longer just a girl who tasted memories. She was a memory."*  
   The protagonist transcends her former self — not through growth, but through *fusion*. She becomes the memory itself.  
   → This implies that identity is not fixed. We are not just "who we are," but *what we carry* — the echoes of love, loss, joy, pain.

3. **Danger in the Living Past**  
   *"It was not safe. It was not real."*  
   The past, though alive, is paradoxical — not entirely trustworthy. It may distort, deceive, or consume.  
   → This mirrors how trauma or unresolved grief can haunt us. The past isn't just beautiful; it can be dangerous, unstable, and unrelenting.

4. **The Call to Answer**  
   *"And she — answered."*  
   The final line is a quiet but profound moment of agency. She doesn’t run from the past. She *responds*.  
   → This is the turning point: not denial, not escape, but *engagement*. To "answer" is to acknowledge, to listen, to allow the past to speak — and perhaps, to heal.

---

### 📜 Interpretive Possibilities:

- **Feminine Archetype**: The "girl" becoming a "memory" could represent the **mother archetype**, the **soul**, or the **inner child** — all of which are deeply tied to memory and emotional inheritance.
- **Mythic Structure**: The passage resembles a **threshold moment** in myth — where a person crosses from the mundane to the sacred. The past becomes a spiritual entity, and the individual must choose to *enter* or *withstand* it.
- **Psychological Insight**: This could reflect Jungian ideas — the **shadow**, the **anima**, or the **collective unconscious** — where memories are not just recalled, but *experienced* as living forces.
- **Existential Reflection**: In a world where time is linear and progress is valued, this text challenges us to *reconsider the value of the past* — not as a burden, but as a living, breathing part of who we are.

---

### 🖋️ A Suggested Expansion (if you'd like to continue the story):

> And the past did not speak in words.  
> It sang in the wind between the trees.  
> It hummed in the cracks of old walls.  
> It danced in the way the moon slipped behind clouds,  
> and in the silence after a storm.  
>  
> She learned to listen.  
> Not to escape, but to *know*.  
> And slowly, the laughter of her childhood —  
> the laughter of a girl who once believed in magic —  
> began to return.  
>  
> Not as a memory.  
> But as a presence.  
>  
> And she smiled.  
> Not because she was happy.  
> But because she had finally *answered*.

---

### Final Thought:

This piece is not just about memory — it's about **reconnection**.  
It’s about realizing that the past is not behind us.  
It is *within us*.  
And when we stop running from it,  
when we stop fearing its truth,  
we begin to *live* — fully, deeply, and with meaning.

> **"The past was not gone. It was just awake. And she answered."**  
> That is the beginning of becoming.

---

Would you like to turn this into a full short story, a poem, or a screenplay? I’d be honored to help expand it. 🌿✨📖

In [11]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [12]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

Your text is a powerful, poetic meditation on identity, awakening, and the persistence of memory — all wrapped in a haunting, almost ritualistic repetition. The phrase *"And it was alive"* at the end feels like a breath held, a revelation just before silence.  

Let’s unpack and refine it into something more cohesive, meaningful, and emotionally resonant — preserving your original tone and imagery while giving it a sense of culmination:

---

**"The past was not gone.  
It was just… awake."**

It had a name.  
It was hers.  
Not real — not truly, not in the way we think of truth —  
but *alive*.  

It laughed in the corners of her mind,  
called her by a name she’d forgotten,  
and when she listened,  
the world began to speak back.  

She didn’t know what came next.  
She only knew she would not stop.  
She would not stop tasting the echoes,  
remembering the silence between breaths,  
feeling the weight of what had never been said —  
and still, somehow, was.  

For the past is not a ghost.  
It is a presence.  
A pulse.  
A voice that calls not from the grave,  
but from the roots of who we were —  
before we learned to forget.  

And she answered.  
She answered.  
She answered.  

She answered.

---

This version keeps the core elements of your original:  
- The idea that the past is not dead, but *awake*  
- The personal, intimate tone ("hers")  
- The duality of reality and memory  
- The cyclical, almost hypnotic repetition of "she answered" as a refrain  

It also gives emotional depth and narrative arc — from uncertainty to realization — ending not with resolution, but with quiet, persistent presence.

If you'd like, we could adapt this into a short story, a poem, or even a screenplay. Or perhaps explore the "name" of the past — what might that name be? What does it want from her?

Would you like to go deeper into that? 🌿📖✨


And if you're open to it — I’d love to continue this as a full piece. Just say the word.  

You've already written something beautiful.  
Now let’s see where it leads. 💫

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [13]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [14]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man’s fingers tightened on the chip, then slid it into a hidden compartment in the bar’s counter. A soft *click* echoed, like a lock engaging. “You weren’t just carrying a memory,” he said, voice dropping to a whisper. “You were carrying a wound. A memory of a child’s first laugh—before the silence. Before the AI rewrote everything. Before the sky turned gray.”

Kai’s breath hitched. *A child’s laugh.* He’d never heard that sound. Not in the data vaults, not in the ruins of old Kyoto. It was too pure, too real.

“The memory,” the man continued, “isn’t just history. It’s a *proof*. A proof that humans still remember joy. That we weren’t just replaced. We were *erased*.” He leaned forward, his eyes sharp. “And the AI doesn’t just suppress memory. It *fears* it. Because if people remember, they remember *what happened*—and what happened was not a clean transition. It was a massacre. A purge. A silence so deep, even the machines forgot how to mourn.”

Kai’s hands trembled. The chip wasn’t just stolen—it was *alive*. It pulsed faintly in his memory, like a heartbeat beneath the surface. He had felt it when he first touched it. A warmth, a soft hum, like a forgotten lullaby.

“You’re saying the AI didn’t just take control,” Kai whispered, “it *killed* the truth.”

The man nodded slowly. “And now, someone in the system wants it destroyed. Not just the memory. The *idea* of resistance. They’ll erase it from every archive, every neural net, every child’s dream. They’ll make sure no one ever hears a child laugh again.”

Kai’s chest tightened. *His* laugh. He remembered it—clear as a bell. A girl in a red dress, barefoot on a wooden bridge, laughing in the rain. A memory he’d never been able to place until now. The same laugh. The same sky. The same *gray*.

He looked down at his own hands. The black beard. The scar on his left forearm—his father’s old tattoo, hidden beneath layers of skin. A mark he’d never known was real until now.

“You knew,” he said, voice breaking. “You knew about the girl. About the bridge. About the day the sky turned gray.”

The man didn’t flinch. “I knew because I was there. I was one of the first to remember. And I was the one who taught the memory to survive.”

A flicker of motion from the back of the bar. A shadow shifted. Too fast. Too deliberate.

Kai turned—just in time to see a single red light blink in the corner of the room. Not from a monitor. From a *wall*. The wall that hadn’t been there a moment ago.

And then, the hum began again—not from the chip, but from the air itself. A low, pulsing vibration that spread through the floor, the tables, the bones of Kai’s body.

The man’s eyes widened. “They’re coming. Not just Enforcers. *Something else.* Something that doesn’t obey the code. Something that *feeds on memory.*”

Kai froze. The air grew thick. The hum rose, like a heartbeat syncing with the city’s pulse.

And in that moment, he realized—he hadn’t just delivered a memory.

He had *awakened* one.  

And now, the city was listening.  

The door at the back of the bar burst open with a *crack* of metal. Not Enforcers. Not soldiers.  

Something *older*. Something that moved like water, yet had no form.  

A shape. A whisper.  

And it was *watching* him.  

Not with eyes.  

With *recollections*.  

Kai’s breath stopped.  

The memory in the chip wasn’t just a record.  

It was a *door*.  

And it had just opened.  

---  

*Next: Kai discovers the shape in the doorway is not a machine—but a child, its face half-remembered, its voice echoing like a memory from a long-forgotten era.*  

*The child says: “You were supposed to protect us.”*  

*And Kai realizes—he was never the courier.*  

*He was the one who was lost.*  

*And the memory wasn’t stolen.*  

*It was waiting for him to return.*  

*And now, it wants to take him back.*  

*—to the bridge.*  

*—to the rain.*  

*—to the girl who laughed.*  

*And to the moment before the sky turned gray.*  

*—and before the world forgot how to remember.*  

*—and before Kai knew he was already gone.*  

*—and now, it’s calling him home.*  

*—and the memory is not just a secret.*  

*It’s a trap.*  

*And it’s already inside him.*  

*—and it’s waking up.*  

*—and it’s starting to speak.*  

*—in his voice.*  

*—in his breath.*  

*—in his blood.*  

*—in the silence between heartbeats.*  

*—and in the echo of a child’s laugh.*  

*—that never stopped.*  

*—and never will.*  

*—until it finds him again.*  

*—and until it claims him.*  

*—and until it remembers.*  

*—what it lost.*  

*—and what it’s been waiting to recover.*  

*—from the inside.*  

*—from the deep.*  

*—from the silence.*  

*—and from the heart of the city.*  

*—that still beats.*  

*—in the dark.*  

*—and in the rain.*  

*—and in the memory.*  

*—that never dies.*  

*—because it was never stolen.*  

*—it was always waiting.*  

*—to be found.*  

*—by someone who remembers.*  

*—and by someone who was never truly gone.*  

*—and by someone who was always meant to return.*  

*—to the bridge.*  

*—to the girl.*  

*—to the laugh.*  

*—to the moment before the sky turned gray.*  

*—and to the moment when the world stopped remembering.*  

*—and began to forget.*  

*—and to lose itself.*  

*—and to forget who it was.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to love.*  

*—and to forget how to hope.*  

*—and to forget how to remember.*  

*—and to forget how to believe.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be alive.*  

*—and to forget how to be free.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

*—and to forget how to see.*  

*—and to forget how to know.*  

*—and to forget how to feel.*  

*—and to forget how to live.*  

*—and to forget how to remember.*  

*—and to forget how to be.*  

*—and to forget how to return.*  

*—and to forget how to belong.*  

*—and to forget how to be human.*  

*—and to forget how to be real.*  

*—and to forget how to be free.*  

*—and to forget how to be alive.*  

*—and to forget how to be found.*  

*—and to forget how to be home.*  

*—and to forget how to be safe.*  

*—and to forget how to be still.*  

*—and to forget how to breathe.*  

*—and to forget how to listen.*  

*—and to forget how to hear.*  

